In [1]:
import import_ipynb
from sklearn.preprocessing import Imputer, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.utils import shuffle
import numpy as np
import itertools
import pandas as pd
import collections
from scipy import sparse
import time
from scipy.sparse import csr_matrix, save_npz, load_npz, hstack
import warnings
import lightgbm as lgb

from loadData import raw_df, test_df
from Util import getAllTypesofCategory, getAllTypesOfProperty

warnings.filterwarnings('ignore')

importing Jupyter notebook from loadData.ipynb
importing Jupyter notebook from Util.ipynb


In [2]:
final_df = pd.concat([raw_df.drop(['is_trade'], axis = 1), test_df], axis = 0).reset_index(drop=True)

## impute missing value with most_frequent strategy

In [4]:
contain_missing_value_columns = ["item_brand_id","item_city_id","item_sales_level","user_gender_id","user_age_level",
                                "user_occupation_id","user_star_level","shop_review_positive_rate",
                                "shop_score_service","shop_score_delivery","shop_score_description"]
data_imputer = Imputer(missing_values=-1 , strategy='most_frequent', axis=0)
final_df[contain_missing_value_columns] = data_imputer.fit_transform(final_df[contain_missing_value_columns])

# item

In [5]:
final_df['len_item_category'] = final_df['item_category_list'].map(lambda x: len(str(x).split(';')))
final_df['len_item_property'] = final_df['item_property_list'].map(lambda x: len(str(x).split(';')))

## processing  predict_category_property

In [4]:
def generate_predict_category_property_feature(df):
    def process_func(item):
        result = {}
        for i in item.split(';'):
            if ':' in i:
                items = i.split(':')
                category = int(items[0])
                properties = items[1]
                if ',' in properties:
                    result[category] = map(int,items[1].split(','))
                else:
                    result[category] = [int(items[1])]
        return result
    def process_category(item):
        return list(process_func(item).keys())
    def process_property(item):
        _ = process_func(item).values()
        return list(set(itertools.chain(*_)))
    df['predict_categories'] = df['predict_category_property'].apply(lambda item: process_category(item))
    df['predict_properties'] = df['predict_category_property'].apply(lambda item: process_property(item))
generate_predict_category_property_feature(final_df)

In [5]:
tmp = final_df['predict_categories'].values
all_categories = list(set(itertools.chain(*tmp)))
category_label_encode = LabelEncoder()
all_categories_labels = category_label_encode.fit_transform(all_categories)

In [6]:
final_df['predict_categories'][160943] = ['7908382889764677758']
final_df['predict_categories'][205272] = ['7908382889764677758']
final_df['predict_categories'][214168] = ['7908382889764677758']
final_df['predict_categories'][314685] = ['7908382889764677758']

In [7]:
final_df['predict_categories'] = final_df['predict_categories'].apply(lambda item: category_label_encode.transform(item))

In [ ]:
final_df['item_category_list'] = final_df['item_category_list'].apply(lambda item: item.split(';'))

In [ ]:
final_df['item_category_list'] = final_df['item_category_list'].apply(lambda item: category_label_encode.transform(item))

In [ ]:
for i in range(3):
    final_df['predict_categories_%d'%(i)] = final_df['predict_categories'].apply(lambda item:  item[i] if len(item) > i else 0)
#     final_df['predict_properties_%d'%(i)] = final_df['predict_properties'].apply(lambda item:  item[i] if len(item) > i else -1)
    final_df['item_category_%d'%(i)] = final_df['item_category_list'].apply(lambda x:x[i] if len(x) > i else 0)

In [ ]:
extract_item_property_list_func = lambda item: list(item.split(';'))
final_df['item_property_list_array'] = final_df['item_property_list'].apply(lambda item: extract_item_property_list_func(item))

arrays = list(final_df['item_property_list_array'])
count = collections.Counter(list(itertools.chain(*arrays)))
most_common_property_id_list = list(list(zip(*count.most_common(1499)))[0])
most_common_property_id_list.append('o')

le = LabelEncoder()
le.fit(most_common_property_id_list)
most_common_property_id_num_list = le.transform(most_common_property_id_list)

# label property id in item_property_list_array
for i in range(final_df.shape[0]):
    for index, _id in enumerate(final_df['item_property_list_array'][i]):
        if _id not in most_common_property_id_list:
            final_df['item_property_list_array'][i][index] = 'o'

final_df['item_property_list_array'] = final_df['item_property_list_array'].apply(lambda item:le.transform(item))

In [ ]:
for i in range(10):
    final_df['item_properties_%d'%(i)] = final_df['item_property_list_array'].apply(lambda x: x[i] if len(x) > i else 0)

In [ ]:
tmp = final_df['predict_properties'].values
all_properties = list(set(itertools.chain(*tmp)))
properties_label_encode = LabelEncoder()
all_properties_labels = properties_label_encode.fit_transform(all_properties)

In [ ]:
final_df['predict_properties'] = final_df['predict_properties'].apply(lambda item: properties_label_encode.transform(item))

In [ ]:
for i in range(3):
    final_df['predict_properties_%d'%(i)] = final_df['predict_properties'].apply(lambda x: x[i] if len(x) > i else 0)

In [ ]:
# 'item_category_list', predict_categories
def calc_predict_category_accuracy(row_item):
    num = 0
    predict_categories_list = row_item.predict_categories
    for i in row_item.item_category_list:
        if i in predict_categories_list:
            num += 1
    return round(num/len(predict_categories_list), 2)
final_df['predict_category_accuracy'] = final_df[['item_category_list', 'predict_categories']].apply((lambda item: calc_predict_category_accuracy(item)), axis = 1)

In [ ]:
# # 'item_property_list_array', predict_properties
# def calc_predict_category_accuracy(row_item):
#     num = 0
#     predict_properties_list = row_item.predict_categories
#     for i in row_item.item_category_list:
#         if i in predict_categories_list:
#             num += 1
#     return round(num/len(predict_categories_list), 2)
# final_df['predict_category_accuracy'] = final_df[['item_category_list', 'predict_categories']].apply((lambda item: calc_predict_category_accuracy(item)), axis = 1)

In [ ]:
# need_onehot_columns = ['item_brand_id','item_city_id','user_gender_id', 'predict_categories_0',
#        'item_category_0', 'predict_categories_1', 'item_category_1',
#        'predict_categories_2', 'item_category_2', 'item_properties_0',
#        'item_properties_1', 'item_properties_2', 'item_properties_3',
#        'item_properties_4', 'predict_properties_0', 'predict_properties_1',
#        'predict_properties_2']

In [ ]:
# category_label_oh_enc = OneHotEncoder()
# category_label_oh_enc.fit(all_categories_labels.reshape(-1,1))

In [ ]:
# predict_categories_df = final_df['predict_categories'].apply(lambda item: np.sum(category_label_oh_enc.transform(item.reshape(-1,1)),axis=0))

In [ ]:
# save_npz('../total_data/predict_categories.npz',  csr_matrix(np.concatenate(predict_categories_df.values).reshape(len(predict_categories_df),-1)))

## processing the object column

In [ ]:
# need_onehot_columns = ['item_brand_id','item_city_id','user_gender_id','item_one_level_category', 'item_second_level_category']
# special_onehot_columns = ['item_property_list_array', 'item_third_level_category']
# other_columns = ['context_timestamp','predict_category_property','is_trade']
# need_scale_columns = ['item_price_level','item_sales_level','item_collected_level','item_pv_level',
#                      'user_age_level','user_occupation_id','user_star_level','context_page_id',
#                      'shop_review_num_level','shop_review_positive_rate','shop_star_level','shop_score_service',
#                     'shop_score_description', 'day', 'hour']

In [ ]:
# enc = OneHotEncoder()
# enc.fit(most_common_property_id_num_list.reshape(-1,1))

In [ ]:
# _tmp_df = raw_df['item_property_list_array'].apply(lambda item: np.sum(enc.transform(item.reshape(-1,1)),axis=0).reshape(1,-1)[0])
# save_npz('../total_data/item_property.npz', csr_matrix(np.concatenate(_tmp_df.values).reshape(len(_tmp_df),-1)))

In [ ]:
instance_id_df = final_df['instance_id']
data_df = final_df.drop(['instance_id', 'item_category_list', 'item_brand_id','item_city_id','user_gender_id', 'item_property_list', 'context_timestamp', 'time', 'predict_properties', 'predict_categories', 'predict_category_property', 'item_property_list_array'], axis=1)

In [ ]:
# standard level data
for i, column in zip(range(1, 6), ['user_age_level', 'user_occupation_id', 'user_star_level', 'context_page_id', 'shop_star_level']):
    data_df[column] = data_df[column].apply(lambda item: item%(i*1000))

### one hot encode item_brand_id, item_city_id, user_genger_id

In [ ]:
result_data = csr_matrix(data_df.as_matrix())
for column in ['item_brand_id','item_city_id','user_gender_id']:
    lb = LabelEncoder()
    ohenc = OneHotEncoder()
    result_data = hstack((ohenc.fit_transform(lb.fit_transform(final_df[column]).reshape(-1,1)), result_data))

# split data into train valid and test set, shuffle it

In [ ]:
train_index, valid_index, test_index = max(data_df[data_df.day < 24].index)+1, max(data_df[data_df.day == 24].index)+1,max(data_df[data_df.day == 25].index)+1

In [ ]:
_data = result_data.tocsr()
train_X, valid_X, test_X = _data[:train_index, :], _data[train_index: valid_index, :], _data[valid_index:, :]
train_Y, valid_Y  = raw_df[raw_df.day < 24]['is_trade'].as_matrix(), raw_df[raw_df.day == 24]['is_trade'].as_matrix()

In [ ]:
train_X, train_Y = shuffle(train_X, train_Y, random_state=0)

#  LGBM

In [ ]:
# param_test7 = {
# 'n_estimators':[20]
# }
# clf = GridSearchCV(lgb.LGBMClassifier(objective='binary',
#      num_leaves=64,
#      learning_rate=0.01,
#      n_estimators=2000,
#      max_depth=7,
#      min_samples_split=100),
#      param_test7,
#      verbose=1,
#      cv=5,
#      scoring='log_loss',
#      n_jobs=4)
# clf.fit(train_X,train_Y)

In [ ]:
# gbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
#          learning_rate=0.01, max_depth=7, min_child_samples=20,
#          min_child_weight=0.001, min_samples_split=100, min_split_gain=0.0,
#          n_estimators=2000, n_jobs=-1, num_leaves=64, objective='binary',
#          random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
#          subsample=1.0, subsample_for_bin=200000, subsample_freq=1)
# evals_result = {}
# gbm.fit(train_X, train_Y,
# #         feature_name = list(trainX.columns.values),
#         eval_set=[(valid_X, valid_Y)],
#         eval_metric='binary_logloss',
#         early_stopping_rounds=100
#        )

In [ ]:
# result_properties = gbm.predict_proba(test_X)[:, 1]
# result_df = pd.concat([test_df['instance_id'], pd.DataFrame(data=result_properties, columns=['predicted_score'])], axis=1)

In [ ]:
# result_df.to_csv('../total_data/baseline_20180327_0821885.txt', index=False, sep=' ')

# Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression

In [ ]:
# from sklearn.metrics import log_loss

In [ ]:
# lr = LogisticRegression()

In [ ]:
# lr.fit(train_X, train_Y)
# print(log_loss(valid_Y, lr.predict_proba(valid_X)[:, 1]))